In [1]:
import requests
import re
import fake_useragent
from bs4 import BeautifulSoup

from os import makedirs

## Константы

In [2]:
STATUS_OK = 200
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
MAIN_URL = f"https://www.kinopoisk.ru/"

## Вспомогательный функции

In [3]:
def get_page_text(
        url: str, 
        user_agent: str, 
        error_log=True
    ) -> str:
    headers = {'user-agent': user_agent}
    response = requests.get(
        url=url,
        headers=headers,
        allow_redirects=True
    )

    if response.status_code == STATUS_OK:
        return response.text
    elif error_log:
        print(f"Error, response code: {response.status_code}\n URL: {url}")

    return None


def clear_link(page_link: str) -> str:
    page_link = re.sub(r"^\/", '', page_link)
    page_link = re.sub(r"[\#\?].+", '', page_link)
    return page_link

def get_page_links(soup: BeautifulSoup, main_url: str) -> set:
    links = set()
    for row in soup.find_all('a'):
        if not row.has_attr("href"):
            continue

        page_link = row.attrs["href"]

        if re.search(r"^(http)", page_link):
            if (main_url not in page_link) or (main_url == page_link):
                continue
        
        page_link = clear_link(page_link=page_link)

        if len(page_link) == 0:
            continue

        links.add(main_url + page_link)

    return links

In [7]:
for page in range(3, 10):
    url = f"https://www.kinopoisk.ru/media/?page={page}"
    page_text = get_page_text(url=url, user_agent=USER_AGENT)

    soup = BeautifulSoup(page_text)
    posts_list = soup.find_all("div", {"class": "posts-grid__main-section-column"})

    for post in posts_list:
        link = post.find("a").attrs["href"]
        title = re.sub('[^А-ЯЁа-яё0-9\s]', ' ', post.text)
        title = re.sub('\s\s+', ' ', title)

        page_url = "https://www.kinopoisk.ru" + link
        clean_path = '/'.join([data for data in page_url.split("/")[2:] if len(data) > 0])

        page_text = get_page_text(page_url, USER_AGENT)

        if page_text is None:
            continue

        dir_path = '../repositories/raw_pages/' + '/'.join(clean_path.split('/')[:-1])
        makedirs(dir_path, exist_ok=True)
        with open(dir_path + '/' + clean_path.split('/')[-1] + '.html', 'w', encoding='utf-8') as f:
            f.write(page_text)

<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/ln/l54hjc713kxgxkw6rfxxgyh00000gn/T/ipykernel_71747/392915334.py:10: SyntaxWarning: invalid escape sequence '\s'
  title = re.sub('[^А-ЯЁа-яё0-9\s]', ' ', post.text)
/var/folders/ln/l54hjc713kxgxkw6rfxxgyh00000gn/T/ipykernel_71747/392915334.py:11: SyntaxWarning: invalid escape sequence '\s'
  title = re.sub('\s\s+', ' ', title)


In [73]:
page_text = get_page_text(url=MAIN_URL, user_agent=USER_AGENT)
soup = BeautifulSoup(page_text)

all_links = get_page_links(soup=soup, main_url=MAIN_URL)
process_link = all_links

while True:
    new_links = set()

    for link in process_link:
        page_text = get_page_text(url=link, user_agent=USER_AGENT)

        soup = BeautifulSoup(page_text)
        links = get_page_links(soup=soup, main_url=MAIN_URL)
        links = links.difference(all_links, new_links)

        new_links = new_links.union(links)

    if len(new_links) == 0:
        break

    all_links = all_links.union(new_links)
    process_links = new_links

Error, response code: 404
 URL: https://www.kinopoisk.ru/media/media/article/4008705/


TypeError: object of type 'NoneType' has no len()